In [301]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [302]:
path="C:\\Users\\best\\Downloads\\Earthquake_damage\\Earthquake_damage"
print(os.listdir(path))

['Building_Ownership_Use.csv', 'Building_Structure.csv', 'ReadMe.md', 'sample_submission.csv', 'test.csv', 'train.csv']


In [303]:
training=pd.read_csv(path+'\\train.csv')      
training.head()
data=training[:1000]
data.head()

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,0,0,0.0,2489
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,0,0,1.0,4423
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,0,0,0.0,3650
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,0,0,0.0,3016
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,0,0,0.0,3627


In [304]:
data=data.drop('area_assesed',axis=1)
data=data.drop('building_id',axis=1)

In [305]:
features=list(data)
print(features)

['damage_grade', 'district_id', 'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement', 'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction', 'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall', 'has_repair_started', 'vdcmun_id']


In [306]:
print(len(data))

1000


In [297]:

print(data.loc[9,"has_repair_started"])


1.0


In [307]:
datacopy=data.copy()
pd.isna(datacopy)


,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False,False,False,False,False
996,False,False,False,False,False,False,False,False,False,False,False,False
997,False,False,False,False,False,False,False,False,False,False,False,False
998,False,False,False,False,False,False,False,False,False,False,False,False


In [263]:
datacopy.fillna(0,inplace=True)

In [310]:
#null_columns=data.columns[data.isnull().any()]
i=0
for i in range(11):
    
    for feature in features:
        if datacopy.loc[i,feature]==True:
            #print("yes")
            data.loc[i,feature]==0
            print(data[i])
        #a=data.loc[i,feature]
        #print(a)
        #print(data[feature].dtype)
        #if data.loc[i,feature]==:
           # print("yes")
        #dataindex=data[data[i][feature] == ''].index
        #print(dataindex)
        #if data[feature].isnull():
            #print(data[i])

#data=data[data.isnull().any(axis=1)][null_columns]==0
#nulldata=data[data.isnull().any(axis=1)][null_columns].head()
#print(nulldata)

KeyError: 1

In [180]:
print(data[data.isnull().any(axis=1)][null_columns].head)

<bound method NDFrame.head of Empty DataFrame
Columns: [has_repair_started]
Index: []>


In [166]:
#i=0
#for i in range(len(data)):
 #   for feature in features:
  #      data.loc[data[feature] == '',feature] =0
            

KeyError: 'area_assesed'

In [272]:
data=data.drop('area_assesed',axis=1)
data=data.drop('building_id',axis=1)
#data['area_assesed'] = pd.to_numeric(data['area_assesed'])
#data['building_id'] = pd.to_numeric(data['building_id'])

In [273]:
x = data.drop('damage_grade', axis=1)
y = data['damage_grade']



In [274]:
data = data.dropna()


In [275]:
from sklearn.model_selection import train_test_split
x_Train, x_Test, y_Train, y_Test = train_test_split(x, y, test_size = 0.2,train_size=0.8)

In [276]:
x_Train.head()

,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
700,22,1.0,0,0,0,1,0,0,1,0.0,2235
606,23,0.0,0,0,0,0,0,0,0,0.0,2356
93,21,0.0,0,0,0,0,0,0,0,0.0,2121
802,30,0.0,0,0,0,0,0,0,0,0.0,3027
501,22,0.0,0,0,0,0,0,0,0,0.0,2234


In [277]:
x_Test.head()

,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
307,29,1.0,1,0,1,1,0,0,0,0.0,2910
786,31,0.0,0,0,0,0,0,0,0,0.0,3143
20,12,0.0,0,0,0,0,0,0,0,0.0,1249
573,25,1.0,0,0,0,0,0,0,1,0.0,2509
439,37,1.0,0,0,0,1,0,0,1,1.0,3723


In [278]:
y_Train.head()

700    Grade 5
606    Grade 5
93     Grade 4
802    Grade 5
501    Grade 5
Name: damage_grade, dtype: object

In [279]:
y_Test.head()

307    Grade 5
786    Grade 2
20     Grade 4
573    Grade 5
439    Grade 5
Name: damage_grade, dtype: object

In [280]:
from sklearn import tree

model = tree.DecisionTreeClassifier()

In [281]:
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [282]:
model.fit(x_Train, y_Train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').